In [ ]:
import os
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models


In [ ]:

# ==== НАСТРОЙКИ ====
DATA_DIR = "data/tl_colors"      # корень с папками red/green/yellow
MODEL_OUT = "tl_color_classifier.pt"
IMG_SIZE = 128
BATCH_SIZE = 64
EPOCHS = 10
LR = 1e-3
VAL_SPLIT = 0.2
NUM_WORKERS = 2

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Трансформации
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


In [ ]:

# Датасет из папок
full_ds = datasets.ImageFolder(DATA_DIR, transform=transform)
class_names = full_ds.classes
print("Classes:", class_names)

val_size = int(len(full_ds) * VAL_SPLIT)
train_size = len(full_ds) - val_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=NUM_WORKERS)

In [ ]:

# Модель: MobileNetV3-small как лёгкий backbone
base_model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT)
in_features = base_model.classifier[-1].in_features
base_model.classifier[-1] = nn.Linear(in_features, len(class_names))
model = base_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [ ]:

# ====================



for epoch in range(EPOCHS):
    # ----- train -----
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # ----- val -----
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = outputs.max(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total

    print(f"Epoch {epoch+1}/{EPOCHS} "
          f"loss={train_loss:.4f} acc={train_acc:.3f} val_acc={val_acc:.3f}")

# Сохранение


In [ ]:
ckpt = {
    "model_state": model.state_dict(),
    "class_names": class_names,
    "img_size": IMG_SIZE,
}
torch.save(ckpt, MODEL_OUT)
print(f"Saved model to {MODEL_OUT}")
